In [0]:
OUTPUTPath = './output/'
!mkdir ./output

In [2]:
# Save model to your Drive
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
import os
import pandas as pd
DATA_SET_DIR = './gdrive/My Drive/NLP/Final-Project/data/'
TRAINING_DATA_PATH = DATA_SET_DIR+'olid-training-clean.csv'
TEST_A_DATA_PATH = DATA_SET_DIR+'testset-levela-clean.tsv'

train = pd.read_csv(TRAINING_DATA_PATH,sep='\t', index_col='id')
test = pd.read_csv(TEST_A_DATA_PATH,sep='\t', index_col='id')
train.head(3)

,tweet_exp,subtask_a,subtask_b,subtask_c
id,,,,
86426,@USER She should ask a few native Americans wh...,OFF,UNT,NaN
90194,@USER @USER Go home you are drunk!!! @USER #MA...,OFF,TIN,IND
16820,Amazon is investigating Chinese employees who ...,NOT,NaN,NaN


In [4]:
print(train.isna().any())
print(test.isna().any())

tweet_exp    False
subtask_a    False
subtask_b     True
subtask_c     True
dtype: bool
tweet_exp    False
dtype: bool


In [5]:
import re, string
def _run_split_on_punc(text):
    """Splits punctuation on a piece of text."""
    chars = list(text)
    i = 0
    start_new_word = True
    output = []
    def _is_punctuation(char):
        # print('[%s]+' % re.escape(string.punctuation))
        if re.match(r'[\!\"\$\%\&\(\)\*\+\,\-\.\/\:\;\<\=\>\?\[\\\]\^_\`\{\|\}\~]+',char):
            return True
        else:
            return False
    while i < len(chars):
        char = chars[i]
        if _is_punctuation(char):
            output.append([char])
            start_new_word = True
        else:
            if start_new_word:
                output.append([])
            start_new_word = False
            output[-1].append(char)
        i += 1
    text_list = ["".join(x) for x in output]
    text = ' '.join(text_list)
    text = text.split(' ')
    return ' '.join(text)
print(_run_split_on_punc("@USER @USER @USER @USER LOL!!!   Throwing the BULLSHIT Flag on such nonsense!!  #PutUpOrShutUp"))

@USER @USER @USER @USER LOL ! ! !    Throwing the BULLSHIT Flag on such nonsense ! !   #PutUpOrShutUp


In [0]:
train.tweet_exp = train.tweet_exp.apply(_run_split_on_punc)
test.tweet_exp = test.tweet_exp.apply(_run_split_on_punc)

In [7]:
import numpy as np
corpus_train = train.tweet_exp
corpus_test = test.tweet_exp
corpus = pd.concat([train.tweet_exp, test.tweet_exp])
print(train.shape,test.shape,corpus.shape)
pd.DataFrame(corpus.iloc[:5],columns=['tweet_exp'])

(13240, 4) (860, 1) (14100,)


,tweet_exp
id,
86426,@USER She should ask a few native Americans wh...
90194,@USER @USER Go home you are drunk ! ! ! @USER...
16820,Amazon is investigating Chinese employees who ...
62688,"@USER Someone should'veTaken "" this piece of ..."
43605,@USER @USER Obama wanted liberals & amp ; il...


In [8]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
MAX_NB_WORDS=10000
tokenizer = Tokenizer(nb_words=None,filters='')
tokenizer.fit_on_texts(corpus)
X = tokenizer.texts_to_sequences(corpus_train)

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/keras_preprocessing/text.py:178: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


In [9]:
for seq in X[:3]:
    print(seq)
    print([tokenizer.index_word[idx] for idx in seq])
MAX_SEQUENCE_LENGTH = max([len(X[i]) for i in range(len(X))])
print('MAX_SEQUENCE_LENGTH:',MAX_SEQUENCE_LENGTH)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

[1, 19, 88, 374, 7, 466, 2798, 313, 39, 70, 134, 25, 24, 4, 2]
['@user', 'she', 'should', 'ask', 'a', 'few', 'native', 'americans', 'what', 'their', 'take', 'on', 'this', 'is', '.']
[1, 1, 102, 329, 6, 10, 979, 9, 9, 9, 1, 56, 1008, 21]
['@user', '@user', 'go', 'home', 'you', 'are', 'drunk', '!', '!', '!', '@user', '#maga', '#trump2020', 'url']
[3410, 4, 6872, 2086, 2564, 51, 10, 1689, 5414, 1526, 5, 1690, 29, 141, 9888, 287, 20, 66, 3411, 17, 3, 3412, 5415, 2, 21, 9889, 56, 417, 4498, 474]
['amazon', 'is', 'investigating', 'chinese', 'employees', 'who', 'are', 'selling', 'internal', 'data', 'to', 'third', '-', 'party', 'sellers', 'looking', 'for', 'an', 'edge', 'in', 'the', 'competitive', 'marketplace', '.', 'url', '#amazon', '#maga', '#kag', '#china', '#tcot']
MAX_SEQUENCE_LENGTH: 113
Found 22043 unique tokens.


In [10]:
import keras
from keras.preprocessing.sequence import pad_sequences
x_training = tokenizer.texts_to_sequences(corpus_train)
x_training = keras.preprocessing.sequence.pad_sequences(x_training,maxlen=MAX_SEQUENCE_LENGTH)
for seq in x_training:
    assert len(seq) == MAX_SEQUENCE_LENGTH
print("all sequence length are ",MAX_SEQUENCE_LENGTH)

all sequence length are  113


In [11]:
# 定義每一個分類對應到的索引數字
label_to_index = {
    'NOT': 0,
    'OFF': 1,
}
# 將分類標籤對應到剛定義的數字
y_train = train.subtask_a.apply(lambda x: label_to_index[x])
y_train.value_counts()

0    8840
1    4400
Name: subtask_a, dtype: int64

In [12]:
y_train = np.asarray(y_train).astype('float32')
print(y_train[:5])

[1. 1. 0. 1. 0.]


In [13]:
from sklearn.model_selection import train_test_split
NUM_LSTM_UNITS = 128
# y_train = keras.utils.to_categorical(y_train)

VALIDATION_RATIO = 0.1
# 小彩蛋
RANDOM_STATE = 9527

x_train, x_val,y_train, y_val =train_test_split(x_training,y_train,test_size=VALIDATION_RATIO,random_state=RANDOM_STATE)
print("Training Set")
print("-" * 10)
print("x_train: ", x_train.shape)
print("y_train : ", y_train.shape)

print("-" * 10)
print("x_val:   ", x_val.shape)
print("y_val :   ", y_val.shape)

Training Set
----------
x_train:  (11916, 113)
y_train :  (11916,)
----------
x_val:    (1324, 113)
y_val :    (1324,)


In [14]:
GLOVE_DIR='./gdrive/My Drive/NLP/Final-Project/WordEmb'
embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.twitter.27B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 1193514 word vectors.


In [15]:
EMBEDDING_DIM = 100
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
null_words = open('./output/null-word.txt', 'w', encoding='utf-8')
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
    else:
        null_words.write(word + '\n')
print('###[EMB] Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

###[EMB] Null word embeddings: 5696


In [0]:
from keras.optimizers import Adam
import pickle
from keras.models import *
from keras.layers import *
from keras.callbacks import ModelCheckpoint

In [17]:
NUM_CLASSES = 2
input_data = Input(shape=(MAX_SEQUENCE_LENGTH,),dtype='int32')
emb_x = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)(input_data)
conv1_x = Conv1D(64, 2, activation='relu')(emb_x)
x_1 =GlobalMaxPooling1D()(conv1_x)
conv2_x = Conv1D(64, 3, activation='relu')(emb_x)
x_2 = GlobalMaxPooling1D()(conv2_x)
conv3_x = Conv1D(64, 5, activation='relu')(emb_x)
x_3 = GlobalMaxPooling1D()(conv3_x)

merged = concatenate([x_1, x_2,x_3],axis=-1)
merged=BatchNormalization()(merged)
merged = Dense(units=300,kernel_initializer='random_normal')(merged)
merged=PReLU()(merged)
merged=BatchNormalization()(merged)
predictions = Dense(1, activation='sigmoid', kernel_initializer='random_normal')(merged)

model = Model(
    inputs=input_data,
    outputs=predictions)
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 113)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 113, 100)     2204400     input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 112, 64)      12864       embedding_1[0][0]                
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 111, 64)      19264       embedding_1[0][0]                
_____________________________________

In [0]:
from keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
class Metrics(Callback):
    def __init__(self, filepath,validation_data=()):
        self.file_path = filepath
        self.X_val, self.y_val = validation_data
    def on_train_begin(self, logs={}):
        self.val_f1s = []
        self.best_val_f1 = 0
        self.val_recalls = []
        self.val_precisions = []

    def on_epoch_end(self, epoch, logs={}):
        val_predict = (np.asarray(self.model.predict(self.X_val))).round()
        val_targ = self.y_val
        _val_f1 = f1_score(val_targ, val_predict,average='macro')
        _val_recall = recall_score(val_targ, val_predict)
        _val_precision = precision_score(val_targ, val_predict)
        self.val_f1s.append(_val_f1)
        self.val_recalls.append(_val_recall)
        self.val_precisions.append(_val_precision)
        print(' — val_f1: %f — val_precision: %f — val_recall %f' %(_val_f1, _val_precision, _val_recall))
        print("max f1")
        print(max(self.val_f1s))
        if _val_f1 > self.best_val_f1:
            self.model.save(self.file_path, overwrite=True)
            self.best_val_f1 = _val_f1
            print("best f1: {}".format(self.best_val_f1))
        else:
            print("val f1: {}, but not the best f1".format(_val_f1))
        return
MODEL_SAVE_PATH='./output/CNN_noTrain.hdf5' 
metrics = Metrics(MODEL_SAVE_PATH,validation_data=(x_val,y_val))

lr = 1e-3
opt = Adam(lr=lr, decay=lr / 50)
model.compile(loss='binary_crossentropy',
          optimizer= "adam")

In [19]:
BATCH_SIZE = 512
NUM_EPOCHS = 50
history = model.fit(
    x=x_train,
    y=y_train,
    batch_size=BATCH_SIZE,
    epochs=NUM_EPOCHS,
    callbacks=[metrics],
    # 每個 epoch 完後計算驗證資料集
    # 上的 Loss 以及準確度
    validation_data=(x_val,y_val),
    # 每個 epoch 隨機調整訓練資料集
    # 裡頭的數據以讓訓練過程更穩定
    shuffle=True,
)

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 11916 samples, validate on 1324 samples
Epoch 1/50
11916/11916 [==============================] - 6s 489us/step - loss: 0.6287 - val_loss: 0.5598
 — val_f1: 0.682587 — val_precision: 0.535373 — val_recall 0.661939
max f1
0.6825870819804085
best f1: 0.6825870819804085
Epoch 2/50
11916/11916 [==============================] - 0s 27us/step - loss: 0.4591 - val_loss: 0.5159
 — val_f1: 0.708076 — val_precision: 0.607748 — val_recall 0.593381
max f1
0.7080758790413723
best f1: 0.7080758790413723
Epoch 3/50
11916/11916 [==============================] - 0s 27us/step - loss: 0.3750 - val_loss: 0.5214
 — val_f1: 0.703079 — val_precision: 0.634078 — val_recall 0.536643
max f1
0.7080758790413723
val f1: 0.7030793613999329, but not the best f1
Epoch 4/50
11916/11916 [==============================] - 0s 27us/step - loss: 0.2959 - val_loss: 0.5372
 — val_f1: 0.71067

In [20]:
x_test = tokenizer.texts_to_sequences(corpus_test)
x_test = keras.preprocessing.sequence.pad_sequences(x_test,maxlen=MAX_SEQUENCE_LENGTH)
for seq in x_test:
    assert len(seq) == MAX_SEQUENCE_LENGTH
print("all sequence length are ",MAX_SEQUENCE_LENGTH)

all sequence length are  113


In [22]:
from keras.models import load_model
if os.path.exists(MODEL_SAVE_PATH):
      model=load_model(MODEL_SAVE_PATH)
    # 若成功加载前面保存的参数，输出下列信息
      print("checkpoint_loaded")
predictions = model.predict(x_test)

checkpoint_loaded


In [24]:
predictions=predictions.round().astype('int').reshape(-1)
predictions[:5]

array([1, 0, 0, 0, 0])

In [28]:
TEST_LABEL_A = DATA_SET_DIR+'labels-levela.csv'
label_a = pd.read_csv(TEST_LABEL_A,sep=',',header=None)
label_a.head(3)

,0,1
0,15923,OFF
1,27014,NOT
2,30530,NOT


In [29]:
y_test_a = label_a[1].apply(lambda x: label_to_index[x])
y_test_a = np.asarray(y_test_a).astype('int')
y_test_a[:5]

array([1, 0, 0, 0, 1])

In [31]:
testing_f1 = f1_score(y_test_a, predictions,average='macro')
print('testing data f1-score:',testing_f1)

testing data f1-score: 0.728026856739728
